In [2]:
#################################################
#    REAL TIME HAND GESTURE DETECTOR            #                        
#    by SUKANYA NASKAR                          #
#    Last Modified - 26th Dec, 2021             #
#################################################

#############################################################################################################
# Prerequisites ----                     
# Python(ver 3.9.0) + OpenCV(ver 4.5.4.60) + Mediapipe(ver 0.8.9) + Numpy(ver 1.20.2) + Tensorflow(ver 2.7.0)
#############################################################################################################

import numpy
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model


## loading mediapipe model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# for webcam input
hands = mp_hands.Hands(model_complexity=0, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)


################ Load the gesture recognizer model
model = load_model('HandGesture_Model')

# Print types of gestures included in this project
with open('list_of_gestures.names', 'r') as f:
    gestures = f.read().split('\n')

    print(("\n\n" + str(gestures)) + "\n\n")


#### Write a loop to capture Camera input 

video_capture = cv2.VideoCapture(0)

while(True):
    if not video_capture.isOpened():
        print("\n\n ENABLE YOUR DEVICE WEBCAM AND START OVER.\n\n")
        break
    else:
        # Store the real time video capture and resize it in desired window size
        success, frame = video_capture.read()
        x, y, c = frame.shape
        # to flip the frame vertically so it takes a mirror image of the user's hand
        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#         frame=cv2.resize(frame, (700, 500),interpolation = cv2.INTER_AREA)

        # Get hand landmark prediction
        result = hands.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        
        # Draw the hand annotations on the image.
        frame.flags.writeable = True
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        op_gesture = ''
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
                for lm in hand_landmarks.landmark:
                    lmx = (lm.x)
                    lmy = (lm.y)
                    landmarks.append([lmx, lmy])
                 
                
                # Detect Gesture
                prediction = model.predict([landmarks])
                
                index = numpy.argmax(prediction)
                
                op_gesture = gestures[index]
                
                
        # show the prediction on the frame
        cv2.putText(frame, op_gesture, (10, 50), cv2.FONT_ITALIC, 1, (255,255,255), 2, cv2.LINE_AA)
                
        cv2.imshow("Let's Test Our Code", frame)
        # press 'x' to exit
        if cv2.waitKey(1) == ord('x'):
            break
        
# When we exit we have to stop the capture and destroy all ative windows
video_capture.release()
cv2.destroyAllWindows()
    



['fist', 'okay']


